In [1]:
import python_version
from python_version import Graph, Tree, Cracker_python
import os

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("pyspark")
sc = SparkContext(conf=conf)

In [3]:
def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    addEdge1 = v_min.cogroup(NN_G_u).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1])))) #if it is possible to reduce to one MapReduce job
    H = addEdge1.flatMap(lambda x: [(x[1][0][0], y) for y in x[1][1][0]]).map(lambda x: (x[1], x[0])).groupByKey().map(lambda x: (x[0], set(x[1])))#.filter(lambda x: len(x[1]) > 1)
    return H

def Pruning_Step(H, T):
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    v_min_filtered = H_filtered.map(lambda x: (x[0], min(x[1])))
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    addEdge2 = v_min_filtered.cogroup(NN_H_u).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
    G = addEdge2.flatMap(lambda x: [(x[1][0][0], y) for y in x[1][1][0]]).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    
    
    #deactiviation
    deactiveNodes = H.filter(lambda x: x[0] not in x[1]).map(lambda x: (x[0], None))
    v_min = H.map(lambda x: (x[0], min(x[1])))
    addEdge3 = deactiveNodes.join(v_min).map(lambda x: (x[1][1], x[0]))
    T = T.union(addEdge3)

    
    return [G, T]

def findSeeds(T):
    T_rev = T.map(lambda x:(x[1], x[0]))
    A = T.keys().distinct().map(lambda x:(x,1))
    B = T_rev.keys().distinct().map(lambda x:(x,1))
    return A.leftOuterJoin(B).filter(lambda x: not x[1][1]).map(lambda x:x[0])

def Cracker_pyspark(G):
    n = 0
    T = sc.parallelize([])
    while G.take(1):
        n += 1
        print(n)
        H = Min_Selection_Step(G)
        G, T = Pruning_Step(H, T)
    
    return T

def Seed_Propragation(T, seed): 
    seed = seed.map(lambda x: (x, x))  
    T_seed = sc.parallelize([(-1, (None, -1))])                       
    
    while T_seed.map(lambda x: (x[1])).lookup(None):
        T_seed = seed.rightOuterJoin(T)
        seed = T_seed.map(lambda x: (x[1][1], x[1][0])).union(seed)
        
    return T_seed

## simulated_blockmodel_graph_100_nodes

In [4]:
os.path.getsize('graph_datasets/simulated_blockmodel_graph_100_nodes.tsv')

6096

In [21]:
G_100 = Graph() 
for line in open('graph_datasets/simulated_blockmodel_graph_100_nodes.tsv',"r"):
    node_1, node_2, c = line.strip().split("\t")
    G_100.addEdge(int(node_1), int(node_2))

T_100 = Cracker_python(G_100)
T_100.seed

{1}

In [30]:
G_100_spark_raw = sc.textFile('graph_datasets/simulated_blockmodel_graph_100_nodes.tsv')
G_100_spark = G_100_spark_raw.map(lambda x: x.split('\t')).map(lambda x: (int(x[0]),int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
G_100_spark.lookup(100)

[{2, 10, 18, 19, 31, 44, 50, 68, 71, 90}]

In [31]:
T_100_spark = Cracker_pyspark(G_100_spark)
findSeeds(T_100_spark).collect()

1
2


[1]

## facebook/107.edges

In [22]:
os.path.getsize('graph_datasets/facebook/107.edges')

523802

In [24]:
G_fb = Graph()
for line in open('graph_datasets/facebook/107.edges',"r"): 
    node_1, node_2 = line.strip().split(" ")
    G_fb.addEdge(int(node_1), int(node_2))
    
T_fb = Cracker_python(G_fb)
T_fb.seed

{0}

In [29]:
G_fb_spark_raw = sc.textFile('graph_datasets/facebook/107.edges')
G_fb_spark = G_100_spark_raw.map(lambda x: x.split(" ")).map(lambda x: (int(x[0]),int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
#G_fb_spark.lookup(960)

In [32]:
T_fb_spark = Cracker_pyspark(G_fb_spark)
findSeeds(T_fb_spark).collect()

1
2
3


[0]

## soc-sign-bitcoinalpha.csv

In [33]:
os.path.getsize('graph_datasets/soc-sign-bitcoinalpha.csv')

503131

In [36]:
G_btc = Graph()
for line in open('graph_datasets/soc-sign-bitcoinalpha.csv',"r"):
    nodes = line.strip().split(",")
    #print(edge)
    G_btc.addEdge(int(nodes[0]), int(nodes[1]))
    
T_btc = Cracker_python(G_btc)
T_btc.seed

{1, 1389, 1870, 3228, 5837}

In [38]:
G_btc_spark_raw = sc.textFile("graph_datasets/soc-sign-bitcoinalpha.csv")
G_btc_spark = G_btc_spark_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
G_btc_spark.lookup(1000)

[{509, 772}]

In [39]:
T_btc_spark = Cracker_pyspark(G_btc_spark)
findSeeds(T_btc_spark).collect()

1
2
3


[1, 1389, 3228, 1870, 5837]

## as-skitter.txt

In [41]:
os.path.getsize('graph_datasets/as-skitter.txt')
#maximum recursion depth exceeded in comparison

149105445

In [46]:
#G_skitter = Graph()
#for line in open('graph_datasets/as-skitter.txt',"r"):
#    nodes = line.strip().split("\t")
    #print(edge)
#    G_skitter.addEdge(int(nodes[0]), int(nodes[1]))
    
#T_skitter = Cracker_python(G_skitter)
#T_skitter.seed

In [52]:
G_skitter_spark_raw = sc.textFile("graph_datasets/as-skitter.txt")
G_skitter_spark = G_skitter_spark_raw.map(lambda x: x.split("\t")).map(lambda x: (int(x[0]), int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
G_skitter_spark.lookup(10000)

[{393, 9997}]

In [53]:
T_skitter_spark = Cracker_pyspark(G_skitter_spark)
findSeeds(T_skitter_spark).collect()

1
2
3
4
5


[0,
 1694463,
 1694498,
 1694500,
 1694506,
 1694511,
 1694518,
 1694524,
 1694527,
 1694531,
 1694534,
 1694539,
 1694543,
 1694546,
 1694548,
 1694553,
 1694555,
 1694561,
 1694563,
 1694579,
 1694584,
 1694588,
 1694590,
 1694593,
 1694600,
 1694602,
 1694605,
 1694608,
 1694610,
 1694612,
 1694622,
 1694627,
 1694629,
 1694633,
 1694635,
 1694638,
 1694640,
 1694642,
 1694644,
 1694646,
 1694656,
 1694658,
 1694660,
 1694663,
 1694665,
 1694669,
 1694673,
 1694675,
 1694677,
 1694679,
 1694682,
 1694685,
 1694688,
 1694691,
 1694694,
 1694697,
 1694699,
 1694704,
 1694712,
 1694714,
 1694716,
 1694719,
 1694721,
 1694723,
 1694725,
 1694728,
 1694732,
 1694734,
 1694736,
 1694740,
 1694742,
 1694744,
 1694746,
 1694748,
 1694751,
 1694757,
 1694759,
 1694763,
 1694765,
 1694767,
 1694769,
 1694771,
 1694773,
 1694775,
 1694777,
 1694779,
 1694782,
 1694784,
 1694786,
 1694788,
 1694791,
 1694793,
 1694795,
 1694797,
 1694800,
 1694802,
 1694804,
 1694806,
 1694809,
 1694811,
 16948